# My Imports

In [75]:
import yfinance as yf
import datetime
import pandas as pd

# Function get_stock_data
## This function will get stock data for given ticker and data + or - 7 days from given date
## The hist dataframe contains data in 30 min intervals for each day
## The daily_hist has data for each day's open, low, high and close

In [79]:
def get_stock_data(ticker, date):
    stock = yf.Ticker(ticker)
    start_date = (date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    end_date = (date + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    
    hist = stock.history(start=start_date, end=end_date, interval='30m')
    daily_hist = stock.history(start=start_date, end=end_date)
    
    return hist, daily_hist

# Function get_close_price
## Gets the close price of a stock for a given date from daily_hist data frame

In [44]:
def get_close_price(daily_hist, date):
    return daily_hist.loc[date.strftime('%Y-%m-%d')]['Close']

# Function get_prior_day
## Gets the prior business day when financial markets was open
## Gets it through available indexes in daily_hist dataframe

In [45]:
def get_prior_day(daily_hist, date):
    prior_day = date - datetime.timedelta(days=1)
    while prior_day.strftime('%Y-%m-%d') not in daily_hist.index.strftime('%Y-%m-%d'):
        prior_day -= datetime.timedelta(days = 1)
        if prior_day < date - datetime.timedelta(days=7):  # Avoid infinite loop
            return None        
    return prior_day

# Function get_next_day
## Gets the prior business day when financial markets was open
## Gets it through available indexes in daily_hist dataframe

In [46]:
def get_next_day(daily_hist, date):
    next_day = date + datetime.timedelta(days=1)
    while next_day.strftime('%Y-%m-%d') not in daily_hist.index.strftime('%Y-%m-%d'):
        next_day += datetime.timedelta(days=1)
        if next_day > date + datetime.timedelta(days=7):  # Avoid infinite loop
            return None         
    return next_day

# Function get_stock_details
## For a given ticker and date, gets all price action, pct increase from prior day, pct increase from today to next day close, strong support indicator, etc..

In [66]:
def get_stock_details(ticker, date):
    hist, daily_hist = get_stock_data(ticker, date)

    if date.strftime('%Y-%m-%d') not in daily_hist.index.strftime('%Y-%m-%d'):
        return None

    prior_day = get_prior_day(daily_hist,date)
    prior_day_close = get_close_price(daily_hist, prior_day)
    

    #Note that time represents start time + delta duration. 
    #For example. if time is 9:30 and delta duration is 60 mins, 9:30 represents data between 9:30 and 10;30
    open_time = f'{date.strftime("%Y-%m-%d")} 09:30:00'
    ten_30_time = f'{date.strftime("%Y-%m-%d")} 10:00:00'
    one_pm_time = f'{date.strftime("%Y-%m-%d")} 12:30:00'
    close_time = f'{date.strftime("%Y-%m-%d")} 15:30:00'
    
    if open_time not in hist.index or ten_30_time not in hist.index or one_pm_time not in hist.index or close_time not in hist.index:
        return None

    open_price = hist.loc[open_time]['Open']
    low_first_hour = hist.loc[open_time:ten_30_time]['Low'].min()
    high_first_hour = hist.loc[open_time:ten_30_time]['High'].max()
    close_first_hour = hist.loc[ten_30_time]['Close']   
    low_till_1pm = hist.loc[open_time:one_pm_time]['Low'].min()
    high_till_1pm = hist.loc[open_time:one_pm_time]['High'].max()
    close_1pm = hist.loc[one_pm_time]['Close']
    close_price = hist.loc[close_time]['Close']
    
    price_movement_since_1pm = close_price - hist.loc[one_pm_time]['Close']

    pct_increase_from_prior_close_to_open = ((open_price - prior_day_close)/prior_day_close)*100
    pct_increase_1pm_to_close = ((close_price - high_till_1pm)/high_till_1pm)*100

    if low_till_1pm >= low_first_hour:
        strong_support = 'Y'
    else:
        strong_support = 'N'

    next_day = get_next_day(daily_hist,date)   
    if next_day is not None:
        next_day_low = daily_hist.loc[next_day.strftime('%Y-%m-%d')]['Low'] 
        next_day_high = daily_hist.loc[next_day.strftime('%Y-%m-%d')]['High'] 
        next_day_close = daily_hist.loc[next_day.strftime('%Y-%m-%d')]['Close']

        pct_increase_next_day_close_to_prior_close = ((next_day_close - close_price)/close_price)*100
    else:
        next_day_low = -1 
        next_day_high = -1 
        next_day_close = -1
        
        pct_increase_next_day_close_to_prior_close = -1
    
    return {
        'Date': date.strftime('%Y-%m-%d'),
        'Ticker': ticker,
        'Prior Close': prior_day_close,
        'Open Price': open_price,
        'Low First Hour': low_first_hour,
        'High First Hour': high_first_hour,
        'Close First Hour': close_first_hour,
        'Low Till 1 PM': low_till_1pm,
        'High Till 1 PM': close_1pm,
        'Close 1 PM': low_till_1pm,
        'Close Price': close_price,
        'Price Movement (1PM - 4PM)': price_movement_since_1pm,
        'Next Day': next_day,
        'Next Day Low': next_day_low,
        'Next Day High': next_day_high,
        'Next Day Close': next_day_close,
        'Pct Increase Prior Close to Open':pct_increase_from_prior_close_to_open,
        'Strong Support': strong_support,
        'Pct Increase 1 PM to Close': pct_increase_1pm_to_close,
        'Pct Increase Close to Next Day Close': pct_increase_next_day_close_to_prior_close,
    }

# Function fetch_price_action_details
## For given list of tickers and number of days, collects all price action details and stores in dataframe

In [61]:
def fetch_price_action_details(tickers, start_date, days=100):
    results = []
    for i in range(days):
        date = start_date - datetime.timedelta(days=i)
        for ticker in tickers:
            data = get_stock_details(ticker, date)
            if data:
                results.append(data)
    
    return pd.DataFrame(results)

# Example Usage

In [115]:

tickers = ['OKTA','HIMS']
start_date = datetime.date(2025, 3, 7)  # Set this to your desired end date
my_stocks_data = fetch_price_action_details(tickers, start_date, days=45)
my_stocks_data

,Date,Ticker,Prior Close,Open Price,Low First Hour,High First Hour,Close First Hour,Low Till 1 PM,High Till 1 PM,Close 1 PM,Close Price,Price Movement (1PM - 4PM),Next Day,Next Day Low,Next Day High,Next Day Close,Pct Increase Prior Close to Open,Strong Support,Pct Increase 1 PM to Close,Pct Increase Close to Next Day Close
0,2025-03-07,OKTA,111.220001,110.519997,110.519997,114.629997,111.699997,107.669998,108.910004,107.669998,112.480003,3.570000,None,-1.000000,-1.000000,-1.000000,-0.629387,N,-1.875594,-1.000000
1,2025-03-07,HIMS,34.389999,34.410000,33.220001,35.549999,34.264114,32.680000,33.460098,32.680000,35.950001,2.489902,None,-1.000000,-1.000000,-1.000000,0.058158,N,1.125180,-1.000000
2,2025-03-06,OKTA,116.309998,114.360001,113.661003,116.959999,115.559998,112.820000,113.290001,112.820000,111.230003,-2.059998,2025-03-07,107.669998,114.629997,112.440002,-1.676551,N,-4.899107,1.087835
3,2025-03-06,HIMS,40.889999,37.400002,36.334801,38.689999,37.720001,35.230000,35.514999,35.230000,34.389999,-1.125000,2025-03-07,32.680000,36.268002,35.950001,-8.535089,N,-11.113981,4.536206
4,2025-03-05,OKTA,108.309998,108.510002,107.375900,112.339897,111.769997,107.375900,112.709999,107.375900,116.349998,3.639999,2025-03-06,111.120003,116.959999,111.220001,0.184659,Y,2.755449,-4.409108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2025-01-24,HIMS,30.889999,31.000000,30.651800,31.770000,31.438499,30.651800,31.410000,30.651800,30.879999,-0.530001,2025-01-27,29.100000,31.870001,30.510000,0.356104,Y,-3.710635,-1.198183
60,2025-01-23,OKTA,88.620003,88.309998,86.305000,88.309998,87.099998,86.305000,86.709999,86.305000,88.070000,1.360001,2025-01-24,87.989998,91.074997,88.860001,-0.349814,Y,-0.271767,0.897015
61,2025-01-23,HIMS,29.719999,29.030001,28.480000,29.590000,29.475000,28.480000,30.049999,28.480000,30.879999,0.830000,2025-01-24,30.652000,32.070000,30.870001,-2.321664,Y,1.947832,-0.032378
62,2025-01-22,OKTA,89.400002,89.580002,88.910004,90.129997,90.069901,88.839996,89.370003,88.839996,88.660004,-0.709999,2025-01-23,86.250000,88.385002,88.070000,0.201343,N,-1.630970,-0.665468


In [113]:
my_stocks_data.columns

Index(['Date', 'Ticker', 'Prior Close', 'Open Price', 'Low First Hour',
       'High First Hour', 'Close First Hour', 'Low Till 1 PM',
       'High Till 1 PM', 'Close 1 PM', 'Close Price',
       'Price Movement (1PM - 4PM)', 'Next Day', 'Next Day Low',
       'Next Day High', 'Next Day Close', 'Pct Increase Prior Close to Open',
       'Strong Support', 'Pct Increase 1 PM to Close',
       'Pct Increase Close to Next Day Close'],
      dtype='object')

In [117]:
my_stocks_data[(my_stocks_data['Pct Increase Prior Close to Open'] >= 3) & (my_stocks_data['Strong Support'] == 'Y')]

,Date,Ticker,Prior Close,Open Price,Low First Hour,High First Hour,Close First Hour,Low Till 1 PM,High Till 1 PM,Close 1 PM,Close Price,Price Movement (1PM - 4PM),Next Day,Next Day Low,Next Day High,Next Day Close,Pct Increase Prior Close to Open,Strong Support,Pct Increase 1 PM to Close,Pct Increase Close to Next Day Close
6,2025-03-04,OKTA,87.160004,100.250000,97.599998,103.879898,102.849998,97.599998,101.629997,97.599998,108.349998,6.720001,2025-03-05,107.375999,116.750000,116.309998,15.018352,Y,3.654453,7.346561
13,2025-02-27,HIMS,41.889999,43.209999,40.480000,45.250000,44.180000,40.480000,44.654999,40.480000,41.314999,-3.340000,2025-02-28,40.139999,45.310001,45.090000,3.151109,Y,-9.792578,9.137121


# Perfecto!!! Bingo!!! Got exactly what I wanted

# Now, let me extend this to all tickers in S&P and QQQ Indexes

In [104]:
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    try:
        table = pd.read_html(url)[0]  # Extract first table from Wikipedia page
        tickers = table['Symbol'].str.replace('.', '-', regex=False).tolist()
        return tickers
    except Exception as e:
        print(f"Error fetching S&P 500 tickers: {e}")
        return []

In [107]:
tickers = get_sp500_tickers()
start_date = datetime.date(2025, 3, 7)  # Set this to your desired end date
my_stocks_data = fetch_price_action_details(tickers, start_date, days=30)
my_stocks_data.head(5)

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [ ]:
my_stocks_data[(my_stocks_data['Pct Increase Prior Close to Open'] >= 4) & (my_stocks_data['Strong Support'] == 'Y')]